<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - KNN Item Item nDCG</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [4]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, item_knn as knn
from lenskit import topn
import pandas as pd
import numpy as np

ratings = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 1M\\ratings.dat', sep='::',engine='python',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [7]:
def model_trainer(neighbours,center):
    neighbours=int(neighbours)
    all_recs = []
    test_data = []
    algo_ii = knn.ItemItem(neighbours,center=center,aggregate='weighted-average')
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test)
        all_recs.append(eval('ItemItem', algo_ii, train, test))
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla = topn.RecListAnalysis()
    rla.add_metric(topn.ndcg)
    results = rla.compute(all_recs, test_data)
    return results["ndcg"].mean()

In [8]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    neighbours = params['neighbours']
    center = params['center']
    metric = model_trainer(neighbours,center)
    print([neighbours,center,metric],',')
    return {'loss': -metric, 'status': STATUS_OK }

space= {'neighbours': hp.uniform('neighbours', 1, 100),
       'center': hp.choice('center', ['True', 'False'])}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

[51.7280652204614, 'True', 0.0729605026727013]                                                                         
,                                                                                                                      
[44.556169615314495, 'True', 0.07338453015863637]                                                                      
,                                                                                                                      
[64.69970586548168, 'True', 0.06965296702485707]                                                                       
,                                                                                                                      
[90.24110622884585, 'True', 0.06834023918694904]                                                                       
,                                                                                                                      
[13.00743624651554, 'False', 0.075668706

KeyboardInterrupt: 

In [9]:
array = [[58.79558499206419, 'False', 0.0711130985155705]         #324.43/55                                                              
,                                                                                                                      
[74.39804017688904, 'False', 0.06977620241902231]                  # 136.47/31                                                    
,                                                                                                                      
[31.93137709010789, 'False', 0.07383694496156969]                                                                      
,                                                                                                                      
[13.420184296991824, 'True', 0.07429450505793697]                                                                      
,                                                                                                                      
[90.7131857478144, 'True', 0.0685584093197583]                                                                         
,                                                                                                                      
[69.7823847137958, 'False', 0.06921178270404095]                                                                       
,                                                                                                                      
[14.034389188091573, 'False', 0.07597723807890727]                                                                     
,                                                                                                                      
[34.330404721789556, 'False', 0.07241611320319942]                                                                     
,                                                                                                                      
[89.18130416923339, 'True', 0.06940815438296896]                                                                       
,                                                                                                                      
[58.25102011702895, 'True', 0.0707225636504331]                                                                        
,                                                                                                                      
[18.52724789894906, 'True', 0.07554056411979904]                                                                       
,                                                                                                                      
[57.15712329799093, 'False', 0.07182802193900997]                                                                      
,                                                                                                                      
[13.06222213717432, 'False', 0.07465781537340795]                                                                      
,                                                                                                                      
[48.11002473624573, 'False', 0.07106044387522177]                                                                      
,                                                                                                                      
[56.77945575225071, 'True', 0.07071550802831175]                                                                       
,                                                                                                                      
[27.209176237132912, 'False', 0.07491108107304048]                                                                     
,                                                                                                                      
[37.80155171552466, 'False', 0.07410219532824112]                                                                      
,                                                                                                                      
[66.80079773513005, 'True', 0.07087294528145918]                                                                       
,                                                                                                                      
[99.94568507193037, 'True', 0.06780728209563235]                                                                       
,                                                                                                                      
[81.43685791512114, 'False', 0.06775411857137616]                                                                      
,                                                                                                                      
[3.3921794583135565, 'True', 0.06168083817748822]                                                                      
,                                                                                                                      
[1.061424793170751, 'True', 0.043690863080274006]                                                                      
,                                                                                                                      
[18.378784540528734, 'True', 0.07547286108443929]                                                                      
,                                                                                                                      
[22.497487577735097, 'True', 0.07480150638772505]                                                                      
,                                                                                                                      
[42.82573760125845, 'True', 0.07301626363135649]                                                                       
,                                                                                                                      
[5.814356684581053, 'False', 0.06875820904322928]                                                                      
,                                                                                                                      
[24.883254306595184, 'False', 0.07531293141157389]                                                                     
,                                                                                                                      
[10.16203522153616, 'True', 0.07563229467497007]                                                                       
,                                                                                                                      
[8.52860266295837, 'False', 0.07382313896734684]                                                                       
,                                                                                                                      
[45.181574089075134, 'True', 0.07202005931074959]                                                                      
,                                                                                                                      
[11.54256689782705, 'False', 0.0731376877147693]                                                                       
,                                                                                                                      
[30.73489509321707, 'False', 0.07416691636601663]                                                                      
,                                                                                                                      
[39.41534528064788, 'True', 0.07379131926752167]                                                                       
,                                                                                                                      
[1.8922064727805914, 'False', 0.043669858965291054]                                                                    
,                                                                                                                      
[18.967661457406706, 'True', 0.07535705933851045]                                                                      
,                                                                                                                      
[51.896792097331826, 'False', 0.07136020932464751]                                                                     
,                                                                                                                      
[31.494217847445015, 'True', 0.07456754111496156]                                                                      
,                                                                                                                      
[15.309208349207712, 'False', 0.07564745910612962]                                                                     
,                                                                                                                      
[63.333446638670054, 'False', 0.07045280155484143]                                                                     
,                                                                                                                      
[77.76673917721516, 'False', 0.06877763265244184]                                                                      
,                                                                                                                      
[36.998185545748704, 'False', 0.07267452555818489]                                                                     
,                                                                                                                      
[18.952067735096836, 'False', 0.07561333247383337]                                                                     
,                                                                                                                      
[52.08056960700605, 'False', 0.07239879727988446]                                                                      
,                                                                                                                      
[16.449190908875135, 'False', 0.0754192887748244]                                                                      
,                                                                                                                      
[28.620559865935903, 'False', 0.07444736166428978]                                                                     
,                                                                                                                      
[24.227465325262838, 'False', 0.0748892012248314]                                                                      
,                                                                                                                      
[5.390307724416646, 'False', 0.06903956248458087]                                                                      
,                                                                                                                      
[34.593041774946784, 'False', 0.07389729946920404]                                                                     
,                                                                                                                      
[98.33088914019785, 'False', 0.06808434606412864]                                                                      
,                                                                                                                      
[14.392882829848789, 'False', 0.07676805692854961]                                                                     
,                                                                                                                      
[7.339372349087003, 'False', 0.07321344631816065]                                                                      
,                                                                                                                      
[41.04103651363822, 'False', 0.07298435496215785]                                                                      
,                                                                                                                      
[72.63382584592595, 'False', 0.07018658940703157]                                                                      
,                                                                                                                      
[22.11642508593909, 'False', 0.0746175071152597]                                                                       
,                                                                                                                      
[63.06835329849868, 'False', 0.07003127228475906]
,
        [43.68404145048749, 'False', 0.0726536189623459]                                                                       
,                                                                                                                      
[17.08548736474902, 'True', 0.07682464322627557]                                                                       
,                                                                                                                      
[56.99486017041279, 'True', 0.07085456499957683]                                                                       
,                                                                                                                      
[30.39194329498944, 'False', 0.07406778456905164]                                                                      
,                                                                                                                      
[38.45666752902646, 'True', 0.07207172029450042]                                                                       
,                                                                                                                      
[18.534281151097623, 'False', 0.07547335170877037]                                                                     
,                                                                                                                      
[97.61171430255148, 'False', 0.06902821938973244]                                                                      
,                                                                                                                      
[1.7264317845668202, 'True', 0.043577595454873455]                                                                     
,                                                                                                                      
[40.01014585296177, 'True', 0.07300533136460986]                                                                       
,                                                                                                                      
[88.54252511717927, 'False', 0.06861209901485962]                                                                      
,                                                                                                                      
[13.17743754817199, 'False', 0.07666370674519576]                                                                      
,                                                                                                                      
[13.927210775805209, 'True', 0.0757492721084145]                                                                       
,                                                                                                                      
[10.18974907214086, 'True', 0.07428120656639961]                                                                       
,                                                                                                                      
[39.241690993347625, 'False', 0.07295390666812063]                                                                     
,                                                                                                                      
[91.5579066916168, 'True', 0.06744487792900382]                                                                        
,                                                                                                                      
[66.17696471822956, 'False', 0.07038364982548466]                                                                      
,                                                                                                                      
[7.684237862845711, 'True', 0.07158266034719549]                                                                       
,                                                                                                                      
[52.607033842521524, 'True', 0.07106149404421258]                                                                      
,                                                                                                                      
[89.8992358703937, 'True', 0.06985462171860937]                                                                        
,                                                                                                                      
[91.33847790167567, 'True', 0.06879533524246192]                                                                       
,                                                                                                                      
[25.133832940786224, 'False', 0.07508565413622155]                                                                     
,                                                                                                                      
[2.9004219787575014, 'False', 0.05445619257918158]                                                                     
,                                                                                                                      
[23.030952913645756, 'False', 0.07372608648486255]                                                                     
,                                                                                                                      
[75.68206651943939, 'False', 0.06884955380812889]                                                                      
,                                                                                                                      
[30.650729337649274, 'False', 0.07356211320478305]                                                                     
,                                                                                                                      
[15.588906007855106, 'True', 0.07521416318945943]                                                                      
,                                                                                                                      
[3.531649992268127, 'True', 0.06239534082285697]                                                                       
,                                                                                                                      
[30.76297497866054, 'False', 0.0753495648824398]                                                                       
,                                                                                                                      
[47.66123712195601, 'False', 0.07132565311380892]                                                                      
,                                                                                                                      
[21.791549690673616, 'True', 0.07531847435147135]                                                                      
,                                                                                                                      
[65.74988151984181, 'False', 0.068401256452305]
,
[51.7280652204614, 'True', 0.0729605026727013]                                                                         
,                                                                                                                      
[44.556169615314495, 'True', 0.07338453015863637]                                                                      
,                                                                                                                      
[64.69970586548168, 'True', 0.06965296702485707]                                                                       
,                                                                                                                      
[90.24110622884585, 'True', 0.06834023918694904]                                                                       
,                                                                                                                      
[13.00743624651554, 'False', 0.07566870610056713]                                                                      
,                                                                                                                      
[43.51706201562345, 'False', 0.07336977291328288]                                                                      
,                                                                                                                      
[22.367804015629893, 'False', 0.07631597117314315]                                                                     
,                                                                                                                      
[89.09943821833384, 'False', 0.06925960660510529]                                                                      
,                                                                                                                      
[25.606880978554557, 'False', 0.07453523350840607]                                                                     
,                                                                                                                      
[91.93141125794244, 'True', 0.06905777533479998]                                                                       
,                                                                                                                      
[54.01655470023174, 'False', 0.0711197950336455]                                                                       
,                                                                                                                      
[13.525557550808749, 'True', 0.07481664569147055]                                                                      
,                                                                                                                      
[24.71514473757149, 'True', 0.0752079957694594]                                                                        
,                                                                                                                      
[99.82011457935927, 'True', 0.06831036426079708]]

In [10]:
metric = pd.DataFrame(array)
metric.columns = ['neighbours','center','nDCG']
metric

,neighbours,center,nDCG
0,58.795585,False,0.071113
1,74.398040,False,0.069776
2,31.931377,False,0.073837
3,13.420184,True,0.074295
4,90.713186,True,0.068558
...,...,...,...
95,91.931411,True,0.069058
96,54.016555,False,0.071120
97,13.525558,True,0.074817
98,24.715145,True,0.075208


In [11]:
metric.sort_values(by=['nDCG'],ascending=False).head(10)

,neighbours,center,nDCG
56,17.085487,True,0.076825
49,14.392883,False,0.076768
65,13.177438,False,0.076664
92,22.367804,False,0.076316
6,14.034389,False,0.075977
66,13.927211,True,0.075749
90,13.007436,False,0.075669
37,15.309208,False,0.075647
27,10.162035,True,0.075632
41,18.952068,False,0.075613


In [12]:
metric.to_csv('metric_nDCG.csv', index=False)